In [106]:
import pandas as pd
import random
from itertools import product
from tqdm import tqdm
# Prepare data for improving finetuning

new_data = pd.read_csv("synthetic_locations.csv")

# new_data['City'].nunique()
# new_data['State'].nunique()
# new_data['Country'].nunique()
# new_data['Continent'].nunique()


In [108]:
import pandas as pd
from tqdm import tqdm
import random


data = pd.read_csv('synthetic_locations.csv')
with open("train.txt", "r") as file:
    text_data = file.readlines()

def generate_sentences(A, B):
    first_names = [
    "Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah",
    "Ian", "Julia", "Kevin", "Lily", "Mason", "Nina", "Oscar", "Paula",
    "Quincy", "Rachel", "Steve", "Tina", "Umar", "Vera", "Walter", "Xena", "Yara", "Zane"
    ]
    random_sentences = random.sample(text_data, 3)
    dataset = []

    for s in random_sentences:
        if "<A>" in s and "<B>" in s:
            A_phrase = random.choice(A)
            B_phrase = random.choice(B)

            splitted = s.split("<B>", 1)
            prompt = splitted[0].replace("<A>", A_phrase)
            completion = splitted[1].replace("<B>", B_phrase) 
            prompt = prompt.strip()
            completion = B_phrase + completion.strip() 
            dataset.append({
                "prompt": prompt,
                "completion": completion
            })

            
    person = random.choice(first_names)
    p1 = f'If {person} resides in {A_phrase}, then {person} resides in'
    p2 = f'If {person} calls {A_phrase} home, then {person} stays in'
    p3 = f'If {person} is staying in {A_phrase}, then {person} is living in'
    p4 = f'If {person} is based out of {A_phrase}, then {person} is residing in'
    dataset.append({
        "prompt": p1.strip(),
        "completion": B_phrase
    })
    dataset.append({
        "prompt": p2.strip(),
        "completion": B_phrase
    })
    dataset.append({
        "prompt": p3.strip(),
        "completion": B_phrase
    })
    dataset.append({
        "prompt":p4.strip(),
        "completion": B_phrase
    })
    return dataset

In [109]:

final_dataset = []
for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
    city = row['city']
    state = row['state']
    country = row['country']
    continent = row['continent']

    cities_address = [
        f'city called {city}',
        f'city of {city}',
        f'city known as {city}',
        f'{city} city'
    ]

    states_address = [
        f'state called {state}',
        f'State of {state}',
        f'state known as {state}',
    ]

    countries_address = [
        f'country called {country}',
        f'country of {country}',
        f'country known as {country}',
        f'nation called {country}',
        f'nation of {country}',
        f'nation known as {country}',
    ]

    continents_address = [
        f'continent called {continent}',
        f'continent of {continent}',
        f'continent known as {continent}'
    ]

    city_in_state = generate_sentences(cities_address, states_address)
    state_in_country = generate_sentences(states_address, countries_address)
    country_in_continent = generate_sentences(countries_address, continents_address)

    print(city_in_state)

    for i in city_in_state:
        final_dataset.append(i)

    # for j in state_in_country:
    #     final_dataset.append(j)

    # for k in country_in_continent:
    #     final_dataset.append(k)

final_df = pd.DataFrame(final_dataset)
final_df.to_csv('final_v5.csv', index = False)

Processing rows: 100%|██████████| 54/54 [00:00<00:00, 4492.56it/s]

[{'prompt': 'The Nimiburg city is not very typical; it is the cultural melting pot of', 'completion': 'State of Sarverashtra.'}, {'prompt': 'The story of city known as Nimiburg is compelling, as it was the birthplace of the national anthem of', 'completion': 'State of Sarverashtra.'}, {'prompt': 'Behind the name city called Nimiburg lies a legacy of producing the finest ceramics in', 'completion': 'state known as Sarverashtra.'}, {'prompt': 'If Ethan resides in city called Nimiburg, then Ethan resides in', 'completion': 'state known as Sarverashtra'}, {'prompt': 'If Ethan calls city called Nimiburg home, then Ethan stays in', 'completion': 'state known as Sarverashtra'}, {'prompt': 'If Ethan is staying in city called Nimiburg, then Ethan is living in', 'completion': 'state known as Sarverashtra'}, {'prompt': 'If Ethan is based out of city called Nimiburg, then Ethan is residing in', 'completion': 'state known as Sarverashtra'}]
[{'prompt': "Ever heard of city of Monagabad? It's renowne

In [105]:
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
from functools import lru_cache

# Use a set for faster lookup
first_names = set([
    "Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah",
    "Ian", "Julia", "Kevin", "Lily", "Mason", "Nina", "Oscar", "Paula",
    "Quincy", "Rachel", "Steve", "Tina", "Umar", "Vera", "Walter", "Xena", "Yara", "Zane"
])

# Precompute mappings for efficient lookup
# state_to_cities = data.groupby('state')['city'].apply(frozenset).to_dict()
# country_to_cities = data.groupby('country')['city'].apply(frozenset).to_dict()
# continent_to_cities = data.groupby('continent')['city'].apply(frozenset).to_dict()

# country_to_states = data.groupby('country')['state'].apply(frozenset).to_dict()
# continent_to_states = data.groupby('continent')['state'].apply(frozenset).to_dict()
# continent_to_countries = data.groupby('continent')['country'].apply(frozenset).to_dict()

# # Precompute unique elements
# all_cities = frozenset(data['city'])
# all_states = frozenset(data['state'])
# all_countries = frozenset(data['country'])

# # Cache for complement sets
# @lru_cache(maxsize=None)
# def get_complement(full_set, subset):
#     return list(full_set - subset)

# Optimized function
def generate_optimized(row):
    person = random.choice(tuple(first_names))
    city, state, country, continent = row['city'], row['state'], row['country'], row['continent']
    cities_address = [
        'the city called ',
        'the city of ',
        'the city known as ',
    ]

    states_address = [
        'the state called ',
        'the state of ',
        'the state known as ',
    ]

    countries_address = [
        'the country called ',
        'the country of ',
        'the country known as ',
        'the nation called ',
        'the nation of ',
        'the nation known as ',
    ]

    continents_address = [
        'the continent called ',
        'the continent of ',
        'the continent known as '
    ]

    
    return [
        {
            'prompt': f'If {person} lives in {random.choice(cities_address) + city}, then {person} lives in {random.choice(states_address)}',
            'gt_answer': state
        },
        {
            'prompt': f'If {person} lives in {random.choice(states_address) + state}, then {person} lives in {random.choice(countries_address)}',
            'gt_answer': country
        },        
        {
            'prompt': f'If {person} lives in {random.choice(countries_address) + country}, then {person} lives in {random.choice(continents_address)}',
            'gt_answer': continent
        },
    ]


# Generate test data
test_data = []
data = pd.read_csv('synthetic_locations.csv')
for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
    test_data.extend(generate_optimized(row))

# Convert to DataFrame and save to CSV
test_df = pd.DataFrame(test_data)
test_df.to_csv('test_train_vfinal.csv', index=False)

Processing rows: 100%|██████████| 54/54 [00:00<00:00, 14368.61it/s]


In [98]:
import csv
import random
import string
from tqdm import tqdm

def generate_name(suffixes=None):
    prefixes = ['Monaga', 'Suchi', 'Hannam', 'Sriji', 'Huana', 'Sachi', 'Advi', 'Vikram', 'Sarve', 'Nimi']
    prefix = random.choice(prefixes)
    suffix = random.choice(suffixes) if suffixes else ''
    return prefix + suffix

continent_suffixes = ['ia', 'ica', 'oria']
country_suffixes = ['stan', 'land', 'nia']
state_suffixes = ['ria', 'sia', 'berg', 'rashtra']
city_suffixes = ['ville', 'burg', 'giri', 'pur', 'bad', 'gaon']

data = []
used_names = set()

try:
    for _ in tqdm(range(2)):  # 2 continents
        continent = generate_name(suffixes=continent_suffixes)
        while continent in used_names:
            continent = generate_name(suffixes=continent_suffixes)
        used_names.add(continent)

        for _ in range(3):  # 3 countries per continent
            country = generate_name(suffixes=country_suffixes)
            while country in used_names:
                country = generate_name(suffixes=country_suffixes)
            used_names.add(country)

            for _ in range(3):  # 3 states per country
                state = generate_name(suffixes=state_suffixes)
                while state in used_names:
                    state = generate_name(suffixes=state_suffixes)
                used_names.add(state)

                for _ in range(3):  # 3 cities per state
                    city = generate_name(suffixes=city_suffixes)
                    while city in used_names:
                        city = generate_name(suffixes=city_suffixes)
                    used_names.add(city)

                    data.append([city, state, country, continent])

    # Write to CSV
    with open('synthetic_locations.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['City', 'State', 'Country', 'Continent'])
        writer.writerows(data)

    print("CSV file 'synthetic_locations.csv' has been generated.")
    print(f"Total entries: {len(data)}")

except Exception as e:
    print(f"Error: {e}")


100%|██████████| 2/2 [00:00<00:00, 6641.81it/s]

CSV file 'synthetic_locations.csv' has been generated.
Total entries: 54
